In [1]:
# define logging and working directory
from ProjectRoot import change_wd_to_project_root
change_wd_to_project_root()
from src.utils.notebook_imports import *
from pyforest import *
from src.utils.utils_io import Console_and_file_logger, ensure_dir
from src.utils.file_wrapper.app.Files import Basefile, Dicomfile
from src.utils.file_wrapper.app.IO import FileWalker
import logging
Console_and_file_logger('all_patients_description_filtered', logging.INFO)

import re
import numpy as np
from scipy import stats
from collections import OrderedDict
import json
import SimpleITK as sitk


search for root_dir and set working directory
Working directory set to: /mnt/data/git/cardio


2020-04-08 12:26:44,985 INFO -------------------- Start --------------------
2020-04-08 12:26:44,986 INFO Working directory: /mnt/data/git/cardio.
2020-04-08 12:26:44,986 INFO Log file: ./logs/all_patients_description_filtered.log


In [2]:
def describe_directory(src_dir=None, description_dir='reports/temp', f_name='temp'):
    """
    Create a json file describing the dicom structure of a given directory
    params: 
    src_dir: path to the dicom root folder (string)
    description_dir: export path for the json files (string)
    f_name: filename for this export (string)
    returns: None
    """

    filewalker = FileWalker(Dicomfile, src_dir=src, dest_dir=description_dir, describe=False)
    ensure_dir(description_dir)
    logging.info('Filewalker done, writing json file...')   

    # save all processed files to disk
    with open('reports/{}.json'.format(f_name), 'w') as outfile:
        json.dump({experiment : filewalker.open_files}, outfile, indent=2)

    # save a json representation of the dataset to disk
    with open('reports/file_walker_{}.json'.format(f_name), 'w') as outfile:
        json.dump({experiment : filewalker.to_json()}, outfile, indent=2)
    logging.info('json dump & file walker for experiment: {} done, everything fine!'.format(f_name))

# Use the Filewalker & the dicom file handler to describe the gcn dataset as json file
src = '/mnt/data/datasets/cardio/ahf_export/0001-02Z1L14N-2006-11-09/'
description_dir = '/mnt/data/git/cardio/data/processed/description'
experiment= 'ahf_description'
describe_directory(src, description_dir, experiment)

In [3]:
import pandas as pd
df = pd.DataFrame.from_dict(filewalker.open_files['Patients'])

NameError: name 'filewalker' is not defined

In [ ]:
df

In [19]:
for patient in 

,PatientAge,PatientBirthDate,PatientID,PatientSex,PatientSize,PatientWeight,Studies,num_images_per_patient
0,not_found,19990601,04NEJQU7,M,not_found,30.0,[{'studyinstanceuid': '1.3.46.670589.11.0.0.11...,401
1,000D,19000101,0AE4R74L,O,not_found,not_found,[{'studyinstanceuid': '1.3.12.2.1107.5.99.2.10...,143
2,not_found,19880101,0HQQW4ZN,F,not_found,45.0,[{'studyinstanceuid': '1.2.124.113532.192.168....,250
3,not_found,19950701,0PTV75MP,F,not_found,43.0,[{'studyinstanceuid': '1.3.46.670589.11.0.0.11...,168
4,not_found,19920601,0RPELLU8,M,not_found,60.0,[{'studyinstanceuid': '1.2.276.0.17.2.998.1913...,325
5,not_found,19861201,13JLP3HN,M,not_found,75,[{'studyinstanceuid': '1.2.276.0.17.2.998.1948...,400
6,not_found,19841001,1DRHF745,F,not_found,51.0,[{'studyinstanceuid': '1.3.46.670589.11.0.0.11...,306
7,not_found,19900301,1E4PF7MR,M,not_found,80.0,[{'studyinstanceuid': '1.3.12.2.1107.5.8.3.485...,308
8,not_found,19900301,1E8MCV9L,F,not_found,59.0,[{'studyinstanceuid': '1.3.46.670589.11.0.0.11...,280
9,not_found,19881201,1EMMCVKN,O,not_found,52.0,[{'studyinstanceuid': '1.3.12.2.1107.5.8.3.485...,240


In [23]:
class IdentDicom(Dicomfile):
    
    def action(self):
        if self.valide():
             return self.to_dcom_json()
        else:
            return {}
    
    def get_metadata_maybe(self, sitk_img, key, default='not_found'):
        try:
            value = sitk_img.GetMetaData(key)
        except Exception as e:
            #print('key not found: {}, {}'.format(key, e))
            value = default
            pass
        return value

    def valide(self):
        # validates if this image is part of a 4d stack
        cardiac_numbers = self.get_metadata_maybe(self.sitk_img,'0018|1090')
        spacing_between_slices = self.get_metadata_maybe(self.sitk_img,'0018|0088')
        
        # first try filter by valid values in both keys
        if cardiac_numbers != 'not_found' and spacing_between_slices != 'not_found':
            if spacing_between_slices == '6 ':
                return True
        else: 
            return False
        return False

In [37]:
# One Filewalker per patient, filter only valid series with IdentDicom
# filter works with D:\\small\\tests\\0006-0AE4R74L-1900-01-01\\ 
# D:\\small\\dcm\\0001-02Z1L14N-2006-11-09\\
# 0002-02Z1L14N-2006-11-09
# 0120-5R8HZG7H-2007-01-08, not working only 3 series
# 0007-0HQQW4ZN-2007-05-23, not working, no key, value
# 0008-0PTV75MP-2005-06-27, not working, no key, value

print(os.getcwd())

experiment= 'ident_small_0007_by_0018,0088'
filewalker = FileWalker(IdentDicom, src_dir='D:\\git\\cardio\\data\\external\\Segmentations\\0007-0HQQW4ZN-2007-05-23\\', dest_dir='D:\\temp\\', describe=False)

logging.info('Filewalker done, writing json file...')   

with open('reports/' + experiment + '.json', 'w') as outfile:
    json.dump({experiment : filewalker.open_files}, outfile, indent=2)
logging.info('json dump {} done, everything fine!'.format(experiment)) 

D:\git\cardio


2018-11-12 13:50:06,847 INFO Action performed in sub-directory D:\git\cardio\data\external\Segmentations\0010-0RPELLU8-2007-02-13\1.3.46.670589.11.0.0.11.4.2.0.21090.5.5268.2007021313513126326 done in 42.351s
2018-11-12 13:50:06,848 INFO Action performed on 500 files
2018-11-12 13:50:06,849 INFO Total Files: 500
2018-11-12 13:50:08,769 INFO Action performed in sub-directory D:\git\cardio\data\external\Segmentations\0010-0RPELLU8-2007-02-13\1.3.46.670589.11.0.0.11.4.2.0.21090.5.5268.2007021314042445903 done in 1.919s
2018-11-12 13:50:08,770 INFO Action performed on 25 files
2018-11-12 13:50:08,771 INFO Total Files: 525
2018-11-12 13:50:13,166 INFO Action performed in sub-directory D:\git\cardio\data\external\Segmentations\0010-0RPELLU8-2007-02-13\1.3.46.670589.11.0.0.11.4.2.0.21090.5.5268.2007021314064396937 done in 4.394s
2018-11-12 13:50:13,167 INFO Action performed on 50 files
2018-11-12 13:50:13,167 INFO Total Files: 575
2018-11-12 13:50:41,130 INFO Action performed in sub-directory

In [38]:
# get all metadata key, values for one dicom image

experiment= 'ident_small_1'
volume_img = IdentDicom(dir_name='D:\\git\\cardio\\data\\processed\\Segmentations\\0006\\0006-0AE4R74L-1900-01-01\\1.3.12.2.1107.5.99.2.1013.30000008030612131778100062857\\', filename='1.3.12.2.1107.5.99.2.1013.30000008030612131778100068703.dcm')

def get_metadata_maybe(sitk_img, key, default='not_found'):
    try:
        value = sitk_img.GetMetaData(key)
    except Exception as e:
        print('key not found: {}, {}'.format(key, e))
        value = default
        pass

    return value

sitk_img = volume_img.sitk_img

keys = sitk_img.GetMetaDataKeys()

for key in keys:
    print(key, get_metadata_maybe(sitk_img, key).encode('utf8','backslashreplace').decode('utf-8').replace('\\udcfc','ue'))
    #print(get_metadata_maybe(sitk_img, key).encode('utf8','backslashreplace').replace('\\udcfc','ue'))
    #print(get_metadata_maybe(sitk_img, key).replace('\\udcfc','ue'))

2018-11-12 14:59:22,808 INFO Error with D:\git\cardio\data\processed\Segmentations\0006\0006-0AE4R74L-1900-01-01\1.3.12.2.1107.5.99.2.1013.30000008030612131778100062857\1.3.12.2.1107.5.99.2.1013.30000008030612131778100068703.dcm: Exception thrown in SimpleITK ReadImage: C:\d\VS14-Win64-pkg\SimpleITK\Code\IO\src\sitkImageReaderBase.cxx:89:
sitk::ERROR: The file "D:\git\cardio\data\processed\Segmentations\0006\0006-0AE4R74L-1900-01-01\1.3.12.2.1107.5.99.2.1013.30000008030612131778100062857\1.3.12.2.1107.5.99.2.1013.30000008030612131778100068703.dcm" does not exist.


UnboundLocalError: local variable 'img' referenced before assignment

In [8]:
experiment= 'ident_small_2'

filewalker = FileWalker(IdentDicom, src_dir='/Users/minority/Code/Git/cardio/data/raw/0001-02Z1L14N-2006-11-09/', dest_dir='D:\\temp\\', describe=False)

#print(json.dumps(filewalker.open_files, indent=2))
with open('reports/' + experiment + '.json', 'w') as outfile:
    json.dump({experiment : filewalker.open_files}, outfile, indent=2)
logging.info('json dump {} done, everything fine!'.format(experiment)) 

2018-11-10 01:08:02,744 INFO Action performed in sub-directory /Users/minority/Code/Git/cardio/data/raw/0001-02Z1L14N-2006-11-09/1.3.12.2.1107.5.99.2.1013.30000008030612131778100065368 done in 0.208s
2018-11-10 01:08:02,748 INFO Action performed on 30 files
2018-11-10 01:08:02,749 INFO Total Files: 30
2018-11-10 01:08:02,889 INFO Action performed in sub-directory /Users/minority/Code/Git/cardio/data/raw/0001-02Z1L14N-2006-11-09/1.3.12.2.1107.5.99.2.1013.30000008030612131778100064249 done in 0.138s
2018-11-10 01:08:02,890 INFO Action performed on 25 files
2018-11-10 01:08:02,891 INFO Total Files: 55
2018-11-10 01:08:02,992 INFO Action performed in sub-directory /Users/minority/Code/Git/cardio/data/raw/0001-02Z1L14N-2006-11-09/1.3.12.2.1107.5.99.2.1013.30000008030612131778100063235 done in 0.100s
2018-11-10 01:08:02,993 INFO Action performed on 18 files
2018-11-10 01:08:02,994 INFO Total Files: 73
2018-11-10 01:08:03,096 INFO Action performed in sub-directory /Users/minority/Code/Git/car